1. metacritic url받아오기
2. /critic-reviews 랑 /user-reviews 내용 가져오기 => url + '/critic-reviews' 
3. 가져 온 텍스트 영어인지 필터 : isalpha()사용? 아님 nltk이용
4. 텍스트로 파일 저장
==> wordcloud

In [2]:
import pandas as pd
import json

from langdetect import detect

import requests
from bs4 import BeautifulSoup

from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from textblob import TextBlob

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import time

from datetime import datetime
from tqdm import tqdm

### 스팀에서 matacritic 정보 받아보기

In [8]:
result = []
rank_length = 200
# 50: 숨겨진 리스트가 50개씩 출력되기 때문
for start in range(0, rank_length, 50):
    URL = 'https://store.steampowered.com/search/results/?query&start='+str(start)+'&count=50&sort_by=Released_DESC&snr=1_7_7_popularnew_7&filter=popularnew&os=win&infinite=1'
    res = requests.get(URL)
    json_data = res.json()
    soup = BeautifulSoup(json_data['results_html'], 'html.parser')
    # steam = soup.select_one('#search_resultsRows')
    titles = soup.select('.col.search_name.ellipsis .title')
    
    for title in titles:
        result.append(title.text)

In [3]:
# setting to get api key outside the Jupyter notebook 
from dotenv import load_dotenv
load_dotenv('.env')

from steam import Steam
import os

KEY = os.getenv('STEAM_API_KEY')  # using env file to access the api key while not exposing it on the code
steam = Steam(KEY)

In [10]:
%%time
url_dict = {}

for game_name in tqdm(result):
    # 데이터 정보가 없는 경우도 있음
    try:
        game = steam.apps.search_games(game_name)
        
        game_id = game['apps'][0]['id']
        game_info = steam.apps.get_app_details(game_id)
        game_info = json.loads(game_info)
        m_url = game_info[str(game_id)]['data']['metacritic']['url']
        url_dict[game_name] =m_url  
    except:
        pass
    
len(url_dict)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [03:11<00:00,  1.04it/s]

CPU times: total: 30.4 s
Wall time: 3min 11s


### url이 스팀 웹에 없는 게임들을 일일이 matacritic 웹사이트에서 검색

#### test 미니 데이터 돌려보기!

In [86]:
# setting driver
path = '../driver/chromedriver.exe'
driver = webdriver.Chrome(service = Service(path))

driver.get('https://www.metacritic.com/game/pc/dave-the-diver/critic-reviews')

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    parent_div = soup.select('div.review_body')
    # 본문 내용 가져오기
    text = []
    for n in range(len(parent_div)):
        if not parent_div[n].find(class_ = 'blurb blurb_expanded') == None:
            content = parent_div[n].find(class_ = 'blurb blurb_expanded').get_text()
            if detect(content) == 'en':
                text.append(content)
            else:
                pass          
        else:
            txt = parent_div[n].get_text()
            if detect(txt) == 'en':
                text.append(txt)
            else:
                pass
            
    # 만들어진 text리스트를 str으로 변환
    textStr = ' '.join(map(str, text))
        
    # word cloud dictionary를 추출!
    stopwords = set(STOPWORDS)
    stopwords.add('game')

    wc = WordCloud(stopwords=stopwords, margin = 10, random_state=1).generate(textStr)
    content = wc.words_
        
    return content

get_content(driver)

driver.close()

### 다수의 게임정보를 추출

In [5]:
# 검색 결과 url을 만들어서 접속하기
def critic_reviews(game_name):
    url = 'https://www.metacritic.com/game/pc/' + game_name + '/critic-reviews'
    # setting driver
    path = '../driver/chromedriver.exe'
    driver = webdriver.Chrome(service = Service(path))
    driver.get(url)
#c_url = critic_reviews(game_name)

def user_reviews(game_name):
    url = 'https://www.metacritic.com/game/pc/' + game_name + '/user-reviews'
    path = '../driver/chromedriver.exe'
    driver = webdriver.Chrome(service = Service(path))
    driver.get(url)
#u_url = critic_reviews(game_name)

# 게시물 내용 가져오기
def get_content(driver):
    # 현재 페이지 html정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    parent_div = soup.select('div.review_body')
    # 본문 내용 가져오기
    text = []
    for n in range(len(parent_div)):
        if not parent_div[n].find(class_ = 'blurb blurb_expanded') == None:
            review = parent_div[n].find(class_ = 'blurb blurb_expanded').get_text()
            if detect(review) == 'en':
                text.append(review)
            else:
                pass          
        else:
            txt = parent_div[n].get_text()
            if detect(txt) == 'en':
                text.append(txt)
            else:
                pass
    # 만들어진 text리스트를 str으로 변환
    textStr = ' '.join(map(str, text))
    
    # word cloud dictionary를 추출!
    stopwords = set(STOPWORDS)
    stopwords.add('game')

    wc = WordCloud(stopwords=stopwords, margin = 10, random_state=1).generate(textStr)
    content = wc.words_
    
    return content


### 만들어진 CLASS와 정보등등으로 FINAL 결과 값을 추출

1. 아애 paht, driver를 class에 넣기
2. time.sleep이 필요한가

In [138]:
results = {}

for game_name in tqdm(game_list):
    try:
        # setting driver
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        # 개별의 게임의 리뷰를 reviews 딕셔너리에 추가
        reviews = {}
        # 개별 게임의 critic 리뷰들을 추출해서 딕셔너리에 추가
        driver.get(critic_reviews(game_name))
        data = get_content(driver)
        reviews['critics'] = data    
        driver.close()
        # 개별 게임의 critic 리뷰들을 추출해서 딕셔너리에 추가
        driver.get(user_reviews(game_name))
        data = get_content(driver)
        reviews['users'] = data
        driver.close()
        
        results[game_name] = reviews['critics'] | reviews['users']
    except:
        time.sleep(5)

100%|██████████| 2/2 [00:10<00:00,  5.01s/it]


In [ ]:
results = {}

for game_name in tqdm(game_list):
    try:
        # setting driver
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        # 개별의 게임의 리뷰를 reviews 딕셔너리에 추가
        reviews = {}
        # 개별 게임의 critic 리뷰들을 추출해서 딕셔너리에 추가
        driver.get(critic_reviews(game_name))
        data = get_content(driver)
        reviews['critics'] = data    
        driver.close()
    except:
        pass


for game_name in tqdm(game_list):
    try:
        # setting driver
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        # 개별의 게임의 리뷰를 reviews 딕셔너리에 추가
        reviews = {}
        # 개별 게임의 user 리뷰들을 추출해서 딕셔너리에 추가
        driver.get(user_reviews(game_name))
        data = get_content(driver)
        reviews['users'] = data
        driver.close()
    except:
        pass

results[game_name] = reviews['critics'] | reviews['users']

In [11]:
results = {}
game_list_t = ['dave-the-diver', 'pizza-tower']
for game_name in tqdm(game_list_t):
    try:
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        
        reviews = {}
        driver.get(critic_reviews(game_name))
        data = get_content(driver)
        reviews['critics'] = data    
        driver.close()
        
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        
        driver.get(user_reviews(game_name))
        data = get_content(driver)
        reviews['users'] = data
        driver.close()
        
        results[game_name] = reviews['critics'] | reviews['users']
    except:
        pass


100%|██████████| 2/2 [01:40<00:00, 50.29s/it]


In [24]:
reviews = {}
results = {}

reviews['critics'] = {'Dave': 0.9230769230769231,
 'Diver': 0.6153846153846154,
 's': 0.5555555555555556,
 'one': 0.46153846153846156,
 'games': 0.38461538461538464,
 'adventure': 0.3076923076923077,
 'player': 0.3888888888888889,
 'characters': 0.3333333333333333}

reviews['users'] = {'sushi': 0.23076923076923078,
 'year': 0.23076923076923078,
 'will': 0.46153846153846156,
 'best': 0.6153846153846154,
 'restaurant': 0.23076923076923078,
 'Even': 0.2222222222222222,
 'many': 0.2222222222222222,
 'loop': 0.3076923076923077}

#results['dave'] = reviews['critics'] | reviews['users']

results['dave'] = reviews['critics'], reviews['users']
results

{'dave': ({'Dave': 0.9230769230769231,
   'Diver': 0.6153846153846154,
   's': 0.5555555555555556,
   'one': 0.46153846153846156,
   'games': 0.38461538461538464,
   'adventure': 0.3076923076923077,
   'player': 0.3888888888888889,
   'characters': 0.3333333333333333},
  {'sushi': 0.23076923076923078,
   'year': 0.23076923076923078,
   'will': 0.46153846153846156,
   'best': 0.6153846153846154,
   'restaurant': 0.23076923076923078,
   'Even': 0.2222222222222222,
   'many': 0.2222222222222222,
   'loop': 0.3076923076923077})}

In [7]:
game_list_t = ['dave-the-diver', 'pizza-tower']
for game_name in tqdm(game_list_t):
    try:
        path = '../driver/chromedriver.exe'
        driver = webdriver.Chrome(service = Service(path))
        reviews = {}
        #print(game_name)
        driver.get(critic_reviews(game_name))
        #time.sleep(120)
        print(critic_reviews(game_name))   
        driver.close()
        
        #results[game_name] = reviews['critics'] | reviews['users']
    except:
        pass

  0%|          | 0/2 [00:00<?, ?it/s]

https://www.metacritic.com/game/pc/dave-the-diver/critic-reviews


 50%|█████     | 1/2 [00:31<00:31, 31.57s/it]

https://www.metacritic.com/game/pc/pizza-tower/critic-reviews


100%|██████████| 2/2 [00:40<00:00, 20.27s/it]


### 게임 리스트가 담긴 csv불러와서 검색에 용이하게 전처리!

In [110]:
df = pd.read_csv('../rank2023.csv')
df = df.loc[0:99,:]

,Rank,Game
0,1,Pizza Tower
1,2,Resident Evil 4
2,3,Pineapple on pizza
3,4,Brotato
4,5,The Murder of Sonic the Hedgehog
...,...,...
195,96,"['Planter', '2', 'Golde', 'Acorn']"
196,97,"['Shogu', 'Showdown']"
197,98,"['W', 'Lov', 'Katamar', 'REROLL', 'Roya', 'Rev..."
198,99,"['EMPT', 'SHELL', 'PROLOGUE']"


In [129]:
game_list = []
for i in df.Game:
    game_name = i.replace(' ', '-').lower()
    game_name = game_name.replace("'", '')
    # 소문자, 특수기호 빼기
    game_list.append(game_name)
    
game_list

['pizza-tower',
 'resident-evil-4',
 'pineapple-on-pizza',
 'brotato',
 'the-murder-of-sonic-the-hedgehog',
 'dave-the-diver',
 'crab-champions',
 'hi-fi-rush',
 'volcano-princess',
 '东北之夏',
 'papas-freezeria-deluxe',
 'villagerhapsody',
 'dredge',
 'aimlabs',
 'steam-engine-simulator',
 'a-space-for-the-unbound',
 'bloody-hell',
 'octopath-traveler-ii',
 'handshakes',
 'hrot',
 'i-commissioned-some-bees-0',
 '候鸟',
 'boneraiser-minions',
 'your-only-move-is-hustle',
 'spin-rhythm-xd',
 'voidigo',
 'hogwarts-legacy',
 'the-outlast-trials',
 'halls-of-torment',
 'ghost-trick:-phantom-detective',
 'cassette-beasts',
 'your-turn-to-die--death-game-by-majority-',
 'barotrauma',
 'touhou\xa0hero\xa0of\xa0ice\xa0fairy',
 'smushi-come-home',
 'staffercase:-a-psychic-mystery-adventure',
 'amanda-the-adventurer',
 'the-test:-secrets-of-the-soul',
 'dr-livesey-rom-and-death-edition',
 'megs-monster',
 'endless-monday:-dreams-and-deadlines',
 'vertigo-2',
 'contraband-police',
 'bramble:-the-mount